In [3]:
from pathlib import Path
temp_path = '../temp'
Path(temp_path).mkdir(exist_ok = 'True')

In [41]:
import sys
sys.path.append('../modules')

In [6]:
!aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/MODELS/TF_Latent_Model_m6_CCM {temp_path+"/TF_Latent_Model_m6_CCM"}
!aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/MODELS/TF_Full_Model_m6_CCM {temp_path+"/TF_Full_Model_m6_CCM"}

download: s3://cdphp-s3-us-e-p-pond/rpi/nneehal/MODELS/TF_Latent_Model_m6_CCM to ../temp/TF_Latent_Model_m6_CCM
download: s3://cdphp-s3-us-e-p-pond/rpi/nneehal/MODELS/TF_Full_Model_m6_CCM to ../temp/TF_Full_Model_m6_CCM


In [8]:
!aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/Reason_AE_m6_large_NNMAtch {temp_path+"/Reason_AE_m6_large_NNMAtch"} --recursive

In [9]:
import numpy as np
import pandas as pd

In [10]:
treated = pd.read_pickle(temp_path+"/Reason_AE_m6_large_NNMAtch/treated0_o.p")
controls = pd.read_pickle(temp_path+"/Reason_AE_m6_large_NNMAtch/match0_o.p")

In [34]:
treated_new = treated.sort_values(by=['PERS_ID', 'MYR']).drop_duplicates(subset=['PERS_ID'], keep='first').where((treated.MYR<=201706)&(treated.MYR>=201701)).dropna()

In [35]:
controls_new = controls.sort_values(by=['PERS_ID', 'MYR']).drop_duplicates(subset=['PERS_ID'], keep='first').where((controls.MYR<=201706)&(controls.MYR>=201701)).dropna()

In [42]:
import tensorflow as tf
from tensorflow.keras import models, layers
import warnings
warnings.filterwarnings('ignore')
from path import col_seq_cc, col_seq_cc_all

In [45]:
model = models.load_model("../temp/TF_Full_Model_m6_CCM", compile=False)

In [49]:
treated_ = treated_new[col_seq_cc_all]
controls_ = controls_new[col_seq_cc_all]

In [53]:
treated_predicted = model.predict(treated_.loc[:,"AGE_AT_MIDMONTH":])
controls_predicted = model.predict(controls_.loc[:,"AGE_AT_MIDMONTH":])

In [58]:
treated_predicted = pd.DataFrame(treated_predicted, columns = treated_.columns.tolist()[2:])
controls_predicted = pd.DataFrame(controls_predicted, columns = controls_.columns.tolist()[2:])

In [69]:
controls_predicted = pd.concat([controls_[["PERS_ID","MYR"]].reset_index(drop=True), controls_predicted], axis=1)
treated_predicted = pd.concat([treated_[["PERS_ID","MYR"]].reset_index(drop=True), controls_predicted], axis=1)

In [70]:
controls_predicted

PERS_ID       MYR  AGE_AT_MIDMONTH  MBR_GNDR  ALLOW_AMT  ALLOW_ER  \
0    2579852  201703.0        63.694576  0.404812   3.174372 -0.015236   
1    2581519  201703.0        64.141640  0.403778   3.200985 -0.016182   
2    2581588  201705.0        58.438969  0.364194   4.377658  0.093578   
3    2589013  201703.0        60.969402  0.398072   3.595314  0.017121   
4    2590057  201703.0        60.557102  0.398382   3.600873  0.019982   
..       ...       ...              ...       ...        ...       ...   
738  4319467  201704.0        30.304028  0.338402   3.196218  0.200959   
739  4319900  201704.0        43.584164  0.413908   2.965846  0.058517   
740  4324430  201706.0        19.774681  0.364118   3.560803  0.295356   
741  4329524  201705.0        33.203056  0.366551   2.989112  0.163331   
742  4362949  201706.0        46.321747  0.412660   3.002694  0.047766   

     ALLOW_IP   CC_ADHD  CC_ALZHEIMER  CC_ALZHEIMER_PLUS  ...  \
0   -0.093402 -0.108686     -0.068482           0.033372  ...   
1   -0.094030 -0.111665     -0.068161           0.035865  ...   
2    0.005050  0.102267     -0.016032           0.137027  ...   
3   -0.061640 -0.042037     -0.042441           0.076430  ...   
4   -0.059054 -0.036383     -0.041558           0.076924  ...   
..        ...       ...           ...                ...  ...   
738  0.009992  0.465961     -0.060655          -0.105941  ...   
739 -0.040952  0.123273     -0.051449          -0.017354  ...   
740  0.066450  1.149659     -0.040872          -0.080809  ...   
741 -0.002677  0.352474     -0.062061          -0.090718  ...   
742 -0.049541  0.091283     -0.053133          -0.009481  ...   

     CC_PROSTATE_CANCER   CC_PTSD  CC_SCHIZOPHRENIA  CC_SCHIZOPHRENIA_OTHER  \
0              0.128486 -0.038995         -0.025935                0.061422   
1              0.130723 -0.039571         -0.026253                0.061138   
2              0.138406  0.080746          0.010714                0.044153   
3              0.138782 -0.009913         -0.017510                0.046387   
4              0.138425 -0.006493         -0.015850                0.047061   
..                  ...       ...               ...                     ...   
738            0.002873  0.250462          0.065619                0.122188   
739            0.066696  0.026166         -0.004839                0.046115   
740           -0.029188  0.375055          0.075688                0.123953   
741            0.011913  0.187912          0.047898                0.108514   
742            0.074719  0.016413         -0.008921                0.046919   

     CC_SPINAL_CORD  CC_SPINA_BIFIDA  CC_STROKE  CC_TOBACCO  \
0         -0.030789        -0.005928  -0.000699    0.227604   
1         -0.030697        -0.005930  -0.000088    0.226648   
2          0.015196        -0.004687   0.038082    0.415405   
3         -0.009783        -0.004210   0.017563    0.266826   
4         -0.008592        -0.003999   0.018213    0.270486   
..              ...              ...        ...         ...   
738       -0.025123         0.001618  -0.065057    0.830747   
739       -0.011653        -0.003756  -0.010869    0.348706   
740       -0.013076         0.013890  -0.036382    0.778276   
741       -0.022639         0.000885  -0.052863    0.696231   
742       -0.014031        -0.004133  -0.009500    0.331712   

     CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  
0             -0.036360         -0.029354  
1             -0.036863         -0.029501  
2             -0.046278         -0.034540  
3             -0.038541         -0.026458  
4             -0.038545         -0.025967  
..                  ...               ...  
738            0.002136         -0.017669  
739           -0.018540         -0.017394  
740            0.011417         -0.010288  
741           -0.001641         -0.016497  
742           -0.020750         -0.018783  

[743 rows x 76 columns]

In [86]:
pd.DataFrame(treated_predicted.mean(axis=0) - controls_predicted.mean(axis=0)).sort_values(by=[0], ascending=False)

0
CC_DX_CNT            0.185609
CC_ANXIETY_DISORDER  0.031610
CC_DEPRESSION        0.024618
ALLOW_AMT            0.019392
CC_OBESITY           0.013881
...                       ...
CC_GLAUCOMA         -0.004326
CC_CATARACT         -0.005176
AGE_AT_MIDMONTH     -0.401096
MYR                       NaN
PERS_ID                   NaN

[76 rows x 1 columns]